In [2]:
import torch
import torch.nn as nn

In [3]:
class Bi_LSTM(nn.Module):
    def __init__(self, config):
        super(Bi_LSTM, self).__init__()
        self.config = config
        self.input_size = config.input_size
        self.hidden_size = config.hidden_size
        self.output_size = config.future_step

        self.bilstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, batch_first=True, bidirectional=True)
        self.bilstm2 = nn.LSTM(input_size=self.hidden_size * 2, hidden_size=self.hidden_size, batch_first=True, bidirectional=True)
        self.linear = nn.Linear(2 * self.hidden_size * self.config.lag_size, self.output_size * self.input_size)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()

    def forward(self, x):
        lstm, (_,_) = self.bilstm(x)
        lstm, (_,_) = self.bilstm2(lstm)
        final = self.linear(self.flatten(lstm))
        final = final.view(self.config.batch_size, self.config.future_step, -1)
        return self.relu(final)

